# Stablecoin Simulation

---

This notebook is part of a stablecoin project.

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Anaylsis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


3. [System Validation](#3.-System-Validation)


---

# 1. System Requirements

<center><img src="images/requirements.png" alt="Requirements" width="70%"/>

## Requirements Analysis

**Goal:** Simulate a simple collateral-backed stablecoin and examine the effects of external price shocks

**Scope:** Learn how simulations can provide insights into specific dynamics

**Question:** How do price shocks affect loan liquidations?

**Assumptions:**
- The price of the underlying loan asset follows a simple stochastic process
- The agents don't follow a certain strategy. They maintain their loan positions arbitrarily
- Liquidations are triggered immediately and go into a sink

## Visual System Mappings

#### Entity Relationship Diagram

TODO

#### Stock & Flow Diagram

TODO

## Mathematical Specification

TODO

---

# 2. System Design

<center><img src="images/design.png" alt="Design" width="70%"/>

## Differential Specification

TODO

## Modelling

In [ ]:
##############
# 0. IMPORTS #
##############

# Standard libraries
import math
from collections import Counter

# Analysis and plotting modules
import pandas as pd
import plotly.express as px

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD stores simulation configuration in the configs list
from cadCAD import configs

In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    'reproduction_rate': [0.01], # sheep per month
    'consumption_rate': [0.01], # tons of grass per month
}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

# Updates run in parallel
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'consumption': p_consumption
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

## Simulation

In [ ]:
####################
# 6. CONFIGURATION #
####################

# the config_sim function creates an object that informs cadCAD about the scope of the simulation
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation
    "T": range(600), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

# Clear any prior configs
del configs[:]

# each cacCAD experiment has configuration options which
# we will pass to the append_configs function to create our simulation configuration
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)


################
# 7. EXECUTION #
################

exec_context = ExecutionContext() # decides how cadCAD should run a simulation (default: single or multi threaded)
simulation = Executor(exec_context=exec_context, configs=experiment.configs)
raw_result, tensor_field, sessions = simulation.execute() # execute the simulation


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)
#simulation_result.head()


##########################
# 9. SIMULATION ANALYSIS #
##########################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

---

# 3. System Validation

<center><img src="images/validation.png" alt="Validation" width="70%"/>